## Test on Properties

This notebook focuses on customizing parsers that can be used to extract information

In this case we are going to try to check for data patching and parsers at the same time. If the extracted properties misalign with compounds extracted, the project would be meaningless.


10/15/2019 meeting 

Challenge:

1. in parser, if a full name with a abbrev in () it will not recognize
2. too messy data paragraphs --> cannot efficiently recognize 


In [1]:
import logging
import re
import pandas as pd
import urllib
import time

import chemdataextractor as cde
from chemdataextractor import Document
from chemdataextractor.model import Compound, UvvisSpectrum, UvvisPeak, BaseModel, StringType, ListType, ModelType
from chemdataextractor.parse.common import hyphen,lbrct, dt, rbrct
from chemdataextractor.parse.base import BaseParser
from chemdataextractor.utils import first

from chemdataextractor.parse.actions import strip_stop
from chemdataextractor.parse.elements import W, I, T, R, Optional, ZeroOrMore, OneOrMore, Or, And
from chemdataextractor.parse.cem import chemical_name,cem, chemical_label, lenient_chemical_label, solvent_name
from chemdataextractor.doc import Paragraph, Sentence, Caption, Figure,Table, Heading
from chemdataextractor.doc.table import Table, Cell

In [2]:
from chemdataextractor.text.chem import SOLVENT_RE, INCHI_RE, SMILES_RE

In [3]:
# open and read files
f = open('../test_articles/paper0.pdf', 'rb')
doc = Document.from_file(f)
abstract = [11]

f1 = open('../test_articles/paper1.pdf', 'rb')
doc1 = Document.from_file(f1)
abstract1 = [7,8]

f2 = open('../test_articles/paper2.pdf', 'rb')
doc2 = Document.from_file(f2)
abstract2 = [7,8]

f3 = open('../test_articles/paper3.pdf', 'rb')
doc3 = Document.from_file(f3)
abstract3 = [10]

f4 = open('../test_articles/paper4.pdf', 'rb')
doc4 = Document.from_file(f4)
abstract4 = [12]

f5 = open('../test_articles/paper5.pdf', 'rb')
doc5 = Document.from_file(f5)
abstract5 = [3,4]

f6 = open('../test_articles/paper6.pdf', 'rb')
doc6 = Document.from_file(f6)
abstract6 = [5,6,7,8]

f7 = open('../test_articles/paper7.pdf', 'rb')
doc7 = Document.from_file(f7)
abstract7 = [11]

In [4]:
# split the paragraph into elements
paras = doc.elements
cems = doc.cems
a = doc.records.serialize()

In [5]:
cems

[Span('PC71BM', 124, 130),
 Span('chloroform', 800, 810),
 Span('monoethanolamine', 588, 604),
 Span('(E)-6,6′-dibromo-1,1′-bis(2-oc-\ntyldodecyl)-[3,3′-biindolinylidene]-2,2′\n-dione', 348, 427),
 Span('nitrogen', 270, 278),
 Span('ITO', 57, 60),
 Span('μ\ne', 0, 3),
 Span('PBDT', 48, 52),
 Span('PBDT', 1840, 1844),
 Span('SiO2', 113, 117),
 Span('PBDT', 304, 308),
 Span('PBDT', 505, 509),
 Span('IIG', 65, 68),
 Span('benzene', 799, 806),
 Span('phenyl-\ncarbazole', 1216, 1233),
 Span('IIG', 226, 229),
 Span('fullerene', 1146, 1155),
 Span('PBDT', 2646, 2650),
 Span('SiO2', 796, 800),
 Span('IIG', 189, 192),
 Span('Naphthalene Diimide-Selenophene', 2014, 2045),
 Span('PBDT', 152, 156),
 Span('PC71BM', 224, 230),
 Span('thiophene', 3889, 3898),
 Span('DIO', 1511, 1514),
 Span('Seki', 1582, 1586),
 Span('Molybdenum\noxide', 191, 207),
 Span('WC\no\nL\n2', 0, 8),
 Span('methanol', 748, 756),
 Span('PBDT', 915, 919),
 Span('PBDT', 2451, 2455),
 Span('1:4\nPBDT-TIIG:PC71BM blend', 181, 207

PCE and FF works fine, as well as other quantities end in %. For other units, further customization required

Most of properties from literature have the same layout, so if one example works, the rest of them should work too.

Any unit with simple expression (1 component) is easy to extract. Otherwise a combination is needed.

An example from CDE. Tg parser's regex phrase is defined in the following code. Our parser should be based on this too.

In [6]:
# All hyphen and minus characters. Probably more robust than the hyph POS tag.
hyphen = R('^[\-‐‑⁃‒–—―−－⁻]$')

# All quote and apostrophe characters
quote = R('^[\'’՚Ꞌꞌ＇‘’‚‛"“”„‟`´’‘]$')

slash = W('/')

In [7]:
from chemdataextractor.model import Compound, UvvisSpectrum, UvvisPeak, BaseModel, StringType, ListType, ModelType
from chemdataextractor.parse.common import hyphen,lbrct, dt, rbrct
from chemdataextractor.parse.base import BaseParser
from chemdataextractor.utils import first
from chemdataextractor.parse.actions import strip_stop, merge, join
from chemdataextractor.parse.elements import W, I, T, R, Optional, ZeroOrMore, OneOrMore, Not, Any
from chemdataextractor.parse.cem import chemical_name
from chemdataextractor.doc import Paragraph, Sentence
from chemdataextractor.model import GlassTransition, Compound

log = logging.getLogger(__name__)

prefix = Optional(I('a')).hide() + (Optional(lbrct) + W('Tg') + Optional(rbrct) | I('glass') + Optional(I('transition')) + Optional((I('temperature') | I('range') | I('temp.'))) | W('transition') + Optional((I('temperature') | I('range') | I('temp.')))
                                    ).hide() + Optional(lbrct + W('Tg') + rbrct) + Optional(W('=') | I('of') | I('was') | I('is') | I('at')).hide() + Optional(I('in') + I('the') + I('range') + Optional(I('of')) | I('about') | ('around') | I('ca') | I('ca.')).hide()

delim = R('^[:;\.,]$')

# TODO: Consider allowing degree symbol to be optional. The prefix should be restrictive enough to stop false positives.
units = (W('°') + Optional(R('^[CFK]\.?$')) |
         W('K\.?'))('units').add_action(merge)

joined_range = R('^[\+\-–−]?\d+(\.\d+)?[\-–−~∼˜]\d+(\.\d+)?$')('value').add_action(merge)

spaced_range = (R('^[\+\-–−]?\d+(\.\d+)?$') + Optional(units).hide() + (R('^[\-–−~∼˜]$') +
                                                                        R('^[\+\-–−]?\d+(\.\d+)?$') | R('^[\+\-–−]\d+(\.\d+)?$')))('value').add_action(merge)

to_range = (R('^[\+\-–−]?\d+(\.\d+)?$') + Optional(units).hide() + (I('to') +
                                                                    R('^[\+\-–−]?\d+(\.\d+)?$') | R('^[\+\-–−]\d+(\.\d+)?$')))('value').add_action(join)

temp_range = (Optional(R('^[\-–−]$')) + (joined_range | spaced_range | to_range))('value').add_action(merge)

temp_value = (Optional(R('^[~∼˜\<\>]$')) + Optional(R('^[\-–−]$')) +
              R('^[\+\-–−]?\d+(\.\d+)?$'))('value').add_action(merge)

temp = Optional(lbrct).hide() + (temp_range | temp_value)('value') + Optional(rbrct).hide()

tg = (prefix + Optional(delim).hide() + temp + units)('tg')

bracket_any = lbrct + OneOrMore(Not(tg) + Not(rbrct) + Any()) + rbrct

cem_tg_phrase = (Optional(cem) + Optional(I('having')).hide() + Optional(delim).hide() + Optional(
    bracket_any).hide() + Optional(delim).hide() + Optional(lbrct) + tg + Optional(rbrct))('tg_phrase')

obtained_tg_phrase = ((cem | chemical_label) + (I('is') | I('are') | I('was')).hide() + (I('measured') |
                                                                                         I('obtained') | I('yielded')).hide() + ZeroOrMore(Not(tg) + Not(cem) + Any()).hide() + tg)('tg_phrase')

#tg_phrase = cem_tg_phrase | method1_phrase | method2_phrase | method3_phrase | obtained_tg_phrase
tg_phrase = cem_tg_phrase | obtained_tg_phrase


class TgParser(BaseParser):
    """"""
    root = tg_phrase

    #print ('outside parser', tg_phrase, type(tg_phrase))

    def interpret(self, result, start, end):
        compound = Compound(
            glass_transitions=[
                GlassTransition(
                    value=first(result.xpath('./tg/value/text()')),
                    units=first(result.xpath('./tg/units/text()'))
                )
            ]
        )
        cem_el = first(result.xpath('./cem'))
        if cem_el is not None:
            compound.names = cem_el.xpath('./name/text()')
            compound.labels = cem_el.xpath('./label/text()')
        yield compound

In [8]:
"""
DataExtractor.parser.voc
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

This is built on top of the chemdataextractor module:
'chemdataextractor.parse.uvvis' as an example for implementaion of the
chemdataextractor pasing syntax.

Parses the open circuit voltage

"""
import logging
import re

from chemdataextractor.model import Compound, UvvisSpectrum, UvvisPeak, BaseModel, StringType, ListType, ModelType
from chemdataextractor.parse.common import hyphen,lbrct, dt, rbrct
from chemdataextractor.parse.base import BaseParser
from chemdataextractor.utils import first
from chemdataextractor.parse.actions import strip_stop
from chemdataextractor.parse.elements import W, I, T, R, Optional, ZeroOrMore, OneOrMore
from chemdataextractor.parse.cem import chemical_name
from chemdataextractor.doc import Paragraph, Sentence

class Voc(BaseModel):
    value = StringType()
    units = StringType()

Compound.voc_pattern = ListType(ModelType(Voc))

abbrv_prefix = (I(u'VOC') | I(u'voc') | I(u'Voc') ).hide()
words_pref = (I(u'open') + I(u'circuit') + I(u'voltage')).hide()
hyphanated_pref = (I(u'open') + hyphen + I('circuit') + I(u'voltage')).hide()

joined_range = R('^[\+\-–−]?\d+(\.\d+)?[\-–−~∼˜]\d+(\.\d+)?$')('value').add_action(merge)
spaced_range = (R('^[\+\-–−]?\d+(\.\d+)?$') + Optional(units).hide() + (R('^[\-–−~∼˜]$') +
                                                                        R('^[\+\-–−]?\d+(\.\d+)?$') | R('^[\+\-–−]\d+(\.\d+)?$')))('value').add_action(merge)
to_range = (R('^[\+\-–−]?\d+(\.\d+)?$') + Optional(units).hide() + (I('to') +
                                                                    R('^[\+\-–−]?\d+(\.\d+)?$') | R('^[\+\-–−]\d+(\.\d+)?$')))('value').add_action(join)



prefix = abbrv_prefix | words_pref | hyphanated_pref
common_text = R('(\w+)?\D(\D+)+(\w+)?').hide()
units = (W(u'V') | I(u'volt'))(u'units')
value = R(u'\d+(\.\d+)?')(u'value')

# voc_first = (prefix + ZeroOrMore(common_text) + value + units)(u'voc')
voc_first = (words_pref + (Optional(lbrct) + abbrv_prefix + Optional(rbrct)) + ZeroOrMore(common_text) + value + units)(u'voc')
voc_second = (value + units + prefix)(u'voc')

voc_pattern = voc_first | voc_second

class VocParser(BaseParser):
    root = voc_pattern

    def interpret(self, result, start, end):
        compound = Compound(
            voc_pattern=[
                Voc(
                    value=first(result.xpath('./value/text()')),
                    units=first(result.xpath('./units/text()'))
                )
            ]
        )
        yield compound


In [9]:
"""
DataExtractor.parsers.pce
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

This is built on top of the chemdataextractor module:
'chemdataextractor.parse.uvvis' as an example for implementaion of the
chemdataextractor pasing syntax.

Parses the power conversion efficiency, it can be applied to solar cells.
PCE is one key parameter to measure the performance of solar cells.

"""

import logging
import re

from chemdataextractor.model import Compound, UvvisSpectrum, UvvisPeak, BaseModel, StringType, ListType, ModelType
from chemdataextractor.parse.common import hyphen
from chemdataextractor.parse.base import BaseParser
from chemdataextractor.utils import first
from chemdataextractor.parse.actions import strip_stop
from chemdataextractor.parse.elements import W, I, T, R, Optional, ZeroOrMore, OneOrMore
from chemdataextractor.parse.cem import chemical_name
from chemdataextractor.doc import Paragraph, Sentence


class Pce(BaseModel):
    value = StringType()
    units = StringType()

Compound.pce_pattern = ListType(ModelType(Pce))

abbrv_prefix = (I(u'PCE') | I(u'PCEs') | I(u'pce')).hide()
words_pref = (I(u'power') + I(u'conversion') + I(u'efficiency')).hide()
hyphanated_pref = (I(u'power') + I(u'-') + I('conversion') + I(u'efficiency')).hide()
prefix = abbrv_prefix | words_pref | hyphanated_pref

common_text = R('(\w+)?\D(\D+)+(\w+)?').hide()
units = (W(u'%') | I(u'percent'))(u'units')
value = R(u'\d+(\.\d+)?')(u'value')

# pce_first  = (words_pref + (Optional(lbrct) + abbrv_prefix + Optional(rbrct)) + ZeroOrMore(common_text) + value + units)(u'pce')

pce_first = (prefix + ZeroOrMore(common_text) + value + units)(u'pce')
pce_second = (value + units + prefix)(u'pce')
pce_pattern = pce_first | pce_second

class PceParser(BaseParser):
    root = pce_pattern

    def interpret(self, result, start, end):
        compound = Compound(
            pce_pattern=[
                Pce(
                    value=first(result.xpath('./value/text()')),
                    units=first(result.xpath('./units/text()'))
                )
            ]
        )
        yield compound

In [10]:
# Sentence.parsers.append(FfParser())
# Paragraph.parsers.append(FfParser())

Sentence.parsers.append(VocParser())
Paragraph.parsers.append(VocParser())

Sentence.parsers.append(PceParser())
Paragraph.parsers.append(PceParser())

In [22]:
doc = Document(
#     Heading('5,10,15,20-Tetra(4-carboxyphenyl)porphyrin (3).'),
#     Paragraph('a glass-transition temperature (Tg) of 20°C'),
#     Paragraph('open circuit voltage (voc) of 0.5 V'),
#     Paragraph('open circuit voltage (voc) as 0.6 V'),
    Paragraph('open circuit voltage (voc) equal to 0.7 V'),
    Paragraph('open circuit voltage (Voc) of 0.8 V'),
    Paragraph('open circuit voltage [voc] of 0.9 V'),
    Paragraph('power conversion efficiency of 10 %'),
    Paragraph('power-conversion efficiency of 12 %'),
#     Paragraph('with the short-circuit current density (Jsc) of 12 mAcm-2'),
#     Paragraph('material with a fill factor (ff) of 0.2'),
)

rec = doc.records.serialize()

In [23]:
rec

[{'voc_pattern': [{'value': '0.7', 'units': 'V'}]},
 {'voc_pattern': [{'value': '0.8', 'units': 'V'}]},
 {'pce_pattern': [{'value': '10', 'units': '%'}]},
 {'pce_pattern': [{'value': '12', 'units': '%'}]}]

In this case we can test on doc7 on data patching. 

In [13]:
doc7[11]

Paragraph(id=None, references=[], text='ABSTRACT: We report the synthesis, properties, and photo-\nvoltaic applications of new π-conjugated polymers having\nthiophene, 3,4-dihexylthiophene, and 1,3,4-oxadiazole (OXD) or\n1,3,4-thiadiazole (TD) units in the main chain, denoted as P1 and\nP2. They were synthesized by the Stille coupling reaction of 2,5-\nbis(trimethylstannyl)thiophene and the corresponding monomers\nof 2,5-bis(5′-bromo-3′,4′-dihexylthien-2′-yl)-1,3,4-oxadiazole or\n2,5-bis(5′-bromo-3′,4′-dihexylthien-2′-yl)-1,3,4-thiadiazole,\nre-\nspectively. The experimental results indicated that the introduc-\ntion of an electron-accepting moiety of OXD or TD lowered the\nhighest occupied molecular orbital (HOMO) energy levels,\nresulting in the higher the open-circuit voltage (Voc) values of\npolymer solar cells (PSCs). Indeed, the PSCs of P1 and P2\nshowed high Voc values in the range 0.80−0.90 V. The highest\nﬁeld-eﬀect transistor (FET) mobilities of P1 and P2 with the OXD and TD moieties, respectively, were 1.41 × 10−3 and 8.81 ×\n10−2 cm2 V−1 s−1. The higher mobility of P2 was related to its orderly nanoﬁbrillar structure, as evidenced from the TEM images.\nMoreover, the higher absorption coeﬃcient and smaller band gap of P2 provided a more eﬃcient light-harvesting ability. The\npower conversion eﬃciency (PCE) of the PSC based on P2:PCBM = 1:1 (w/w) reached 3.04% with a short-circuit current\ndensity (Jsc) value of 6.60 mA/cm2, a Voc value of 0.80 V, and a ﬁll factor (FF) value of 0.576 during the illumination of AM 1.5,\n100 mW/cm2. In comparison,\nthe electron-accepting moiety exhibited an inferior device\nperformance (FET mobility = 2.10 × 10−4 cm2 V−1 s−1 and PCE = 1.91%). The experimental results demonstrated that\nincorporating the electron-acceptor moiety into the polythiophene backbone could enhance the device performance due to the\nlow-lying HOMO levels, compact packing structure, and high charge carrier mobility. This is the ﬁrst report for the achievement\nof PCE > 3% using PSCs based on polythiophenes having TD units in the main chain.')

In [14]:
doc = Document(
    Heading('Abstract:'),
    Paragraph('We report the synthesis, properties, and photo- voltaic applications of new π-conjugated polymers having thiophene, 3,4-dihexylthiophene, and 1,3,4-oxadiazole (OXD) or 1,3,4-thiadiazole (TD) units in the main chain, denoted as P1 and P2. They were synthesized by the Stille coupling reaction of 2,5- bis(trimethylstannyl)thiophene and the corresponding monomers of 2,5-bis(5′-bromo-3′,4′-dihexylthien-2′-yl)-1,3,4-oxadiazole or 2,5-bis(5′-bromo-3′,4′-dihexylthien-2′-yl)-1,3,4-thiadiazole, re- spectively. '),
    Paragraph('The experimental results indicated that the introduc- tion of an electron-accepting moiety of OXD or TD lowered the highest occupied molecular orbital (HOMO) energy levels, resulting in the higher the open-circuit voltage (Voc) values of polymer solar cells (PSCs). Indeed, the PSCs of P1 and P2 showed high Voc values in the range 0.80−0.90 V. The highest ﬁeld-eﬀect transistor (FET) mobilities of P1 and P2 with the OXD and TD moieties, respectively, were 1.41 × 10−3 and 8.81 × 10−2 cm2 V−1 s−1. '),
    Paragraph('The higher mobility of P2 was related to its orderly nanoﬁbrillar structure, as evidenced from the TEM images. Moreover, the higher absorption coeﬃcient and smaller band gap of P2 provided a more eﬃcient light-harvesting ability. '),
    Paragraph('The power conversion eﬃciency (PCE) of the PSC based on P2:PCBM = 1:1 (w/w) reached 3.04 % with a short-circuit current density (Jsc) value of 6.60 mA/cm2, a Voc value of 0.80 V, and a fill factor value of 57.6% during the illumination of AM 1.5, 100 mW/cm2. '),
    Paragraph('In comparison, the electron-accepting moiety exhibited an inferior device performance (FET mobility = 2.10 × 10−4 cm2 V−1 s−1 and PCE = 1.91%). The experimental results demonstrated that incorporating the electron-acceptor moiety into the polythiophene backbone could enhance the device performance due to the low-lying HOMO levels, compact packing structure, and high charge carrier mobility. This is the ﬁrst report for the achievement of PCE > 3% using PSCs based on polythiophenes having TD units in the main chain.')
)

rec = doc.records.serialize()

### Example to run on the abstract

In [15]:
abstract1_doc = doc1.elements[7] + doc1.elements[8]

In [16]:
abstract1_doc

Paragraph(id=None, references=[], text='Conjugated copolymers based on isoindigo/fluorinated isoindigo and\nbis(dialkylthienyl)benzodithiophene, PBDTT–ID and PBDTT–FID, were\ndesigned and synthesized for organic photovoltaic applications. The\nisoindigo fluorination made PBDTT–FID show high PCE of 5.52% and\n7.04% in its conventional and inverted PSCs, respectively.Recently, tremendous eﬀorts on low band gap (LBG) copolymers and\ntheir polymer solar cells (PSCs) have been devoted to improve the\npower conversion eﬃciencies (PCEs) from 1% to over 10%.1 Iso-\nindigo (ID) was newly developed as an eﬃcient acceptor block due to\nthe strong electron-withdrawing character and large optical transition\ndipole of two lactam rings. Reynolds et al. reported the first ID-based\nphotovoltaic oligomer, exhibiting a PCE of 1.76%.2 Andersson et al.\nsynthesized a new ID-based copolymer, PTI-1, using thiophene as a\ndonor block with designed side chains. PSCs fabricated from PTI-1\nand PC71BM as the photoactive layer showed a PCE of 6.3%.3 Other\nID-based copolymers containing diﬀerent donor blocks, such as\ndithienosilole, benzodithiophene, naphthalene, anthracene and\ncyclopentadithiophene, were also synthesized and investigated later.4\nThe ID-based polymers reported so far exhibit broad absorption and\nthe low-lying HOMO (\x005.50 to \x005.90 V) and LUMO (\x003.70 to\n\x004.00 V) with low band gaps ranging from 1.3 to 1.8 eV. In addition,\nthey possess high carrier mobilities. Pei et al. reported ID-based\nconjugated polymers with hole mobility up to 1.06 cm2 V\x001 s\x001.5 Bao\net al. introduced siloxane-terminated solubilizing groups on the\nisoindigo skeleton for shortening the p–p stacking distance, which\nimproved the hole mobility to 2.48 cm2 V \x001 s\x001.6 After this, Pei et al.\nreported another fluorinated ID-based polymer PFII2T with good\nambient stability. The electron mobility increased from 10\x002 to\n0.43 cm2 V\x001 s\x001 with high hole mobility up to 1.85 cm2 V \x001 s\x001.7\nIn this communication, we firstly reported the design and synth-\nesis of the D–A photovoltaic copolymers of isoindigo (ID)/fluorinated')

In [17]:
abstract1_doc.records.serialize()

[{'names': ['isoindigo']},
 {'names': ['bis(dialkylthienyl)benzodithiophene']},
 {'names': ['PBDTT']},
 {'names': ['isoindigo']},
 {'names': ['transition dipole of two lactam']},
 {'pce_pattern': [{'value': '5.52', 'units': '%'}]},
 {'names': ['thiophene']},
 {'pce_pattern': [{'value': '1.76', 'units': '%'}]},
 {'names': ['dithienosilole']},
 {'names': ['benzodithiophene']},
 {'names': ['naphthalene']},
 {'names': ['anthracene']},
 {'names': ['cyclopentadithiophene']},
 {'pce_pattern': [{'value': '6.3', 'units': '%'}]},
 {'names': ['siloxane']},
 {'names': ['isoindigo']},
 {'names': ['fluorinated ID']},
 {'names': ['isoindigo', 'ID']}]